In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
# importing the data
data = pd.read_csv("FoDS-A1.csv", names = ["X1","X2","Y"], header=0)

# Data Preprocessing

Preprocessing the data involves:

- `Normalizing` the data:<br>
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; <font size="4"> X = $\frac{ X_{} - X_{min} }{X_{max} - X_{min}}$ </font>
<br>
<br>                 
- `Shuffling` the data
<br>
<br>
- `Splitting` the data to test and train 

In [58]:
def normalize(data):
    data_min = data.min()
    data_max = data.max()
    data["X1"] = (data["X1"] - data_min[0])/(data_max[0]-data_min[0])
    data["X2"] = (data["X2"] - data_min[1])/(data_max[1]-data_min[1])

In [59]:
def split(data,fraction = 0.7):
    
    shuffled_data = data.sample(frac=1, random_state=0) # Shuffling the dataset
    split_index = int(fraction * len(data)) # Finding split index
    
    # Spliting the dataset 
    train= shuffled_data[:split_index]
    test = shuffled_data[split_index:]
    return train.reset_index(drop=True), test.reset_index(drop=True)

In [60]:
def generate_vectors(train_X, degree=9):
    total_terms = int(((degree+1)*(degree+2))/2)
    terms = np.zeros((total_terms,train_X.shape[0]))
    
    
    for i in range(train_X.shape[0]):

        current_x1 = train_X[i][0]
        current_x2 = train_X[i][1]
        m = 0
        part_x1 = 1
        part_x2 = 1
        
        for j in range(degree+1):
            for k in range(degree-j+1):
                terms[m][i] = part_x1 * part_x2
                m += 1
                part_x1 = part_x1 * current_x1 
            
            part_x2 = part_x2 * current_x2
            part_x1 = 1
            
    W = np.zeros((total_terms,1))
                
    return terms.T, W

In [73]:
def calculate_cost_ridge(W, X, Y, penalty):
    cost = (np.sum(np.square(np.dot(X,W) - Y)))/2
    cost += (penalty * np.dot(W[1:].T, W[1:])[0][0])/2
    return cost

In [62]:
def calculate_cost_lasso(W, X, Y, penalty):
    cost = (np.sum(np.square(np.dot(X,W) - Y)))/2
    cost += penalty * np.sum(np.absolute(W[1:]))
    return cost

In [63]:
def gradient_descent_ridge(X, Y, W, penalty, learning_rate=0.01, iterations=1000):

    cost_history = np.zeros(iterations)
    for it in range(iterations):
        
        prediction = np.dot(X, W)         
        updated_W0 = W[0] - np.multiply((X.T.dot(prediction - Y)), learning_rate)[0]
        W[1:] = W[1:] - np.multiply((X.T.dot(prediction - Y)[1:] + np.multiply(W[1:], penalty)), learning_rate)
        W[0] = updated_W0
        cost_history[it]  = calculate_cost_ridge(W, X, Y, penalty)
        
    return W, cost_history

In [94]:
def gradient_descent_lasso(X, Y, W,  penalty, learning_rate=0.0001, iterations=1000):

    cost_history = np.zeros(iterations)
    for it in range(iterations):
        
        prediction = np.dot(X, W)         
        updated_W0 = W[0] - np.multiply((X.T.dot(prediction - Y)), learning_rate)[0]
        W[1:] = W[1:] - np.multiply((X.T.dot(prediction - Y)[1:] + np.multiply(np.divide(W[1:], np.absolute(W[1:])), penalty)), learning_rate)
        W[0] = updated_W0
        cost_history[it]  = calculate_cost_lasso(W, X, Y, penalty)
        
    return W, cost_history

In [99]:
def stochastic_gradient_descent_ridge(X, Y, W, penalty, learning_rate=0.01, iterations=10):

    m = len(Y)
    m = int(m / 10)
    cost_history = np.zeros(iterations)
    
    for it in range(iterations):
        cost = 0.0
        for i in range(m):
            rand_int = np.random.randint(0,m)
            X_i = X[rand_int:rand_int+1]
            Y_i = Y[rand_int:rand_int+1]

            prediction = np.dot(X_i,W)
            updated_W0 = W[0] - np.multiply((X_i.T.dot(prediction - Y_i)), learning_rate)[0]
            W[1:] = W[1:] - np.multiply((X_i.T.dot(prediction - Y_i)[1:] + np.multiply(W[1:], penalty)), learning_rate)
            W[0] = updated_W0
            
            cost += calculate_cost_ridge(W,X_i,Y_i, penalty)
        cost_history[it]  = cost
        
    return W, cost_history

In [98]:
def stochastic_gradient_descent_lasso(X, Y, W, penalty, learning_rate=0.01, iterations=10):

    m = len(Y)
    m = int(m / 10)
    cost_history = np.zeros(iterations)
    
    for it in range(iterations):
        cost =0.0
        for i in range(m):
            rand_int = np.random.randint(0,m)
            X_i = X[rand_int:rand_int+1]
            Y_i = Y[rand_int:rand_int+1]

            prediction = np.dot(X_i,W)
            updated_W0 = W[0] - np.multiply((X_i.T.dot(prediction - Y_i)), learning_rate)[0]
            W[1:] = W[1:] - np.multiply((X_i.T.dot(prediction - Y_i)[1:] + np.multiply(np.divide(W[1:], np.absolute(W[1:])), penalty)), learning_rate)
            W[0] = updated_W0
            print(W)
            
            cost += calculate_cost_lasso(W,X_i,Y_i, penalty)
        cost_history[it]  = cost
        
    return W, cost_history

In [67]:
def errorloop(cost_history):
    for it in range(len(cost_history)):
        if it%50==0: print(cost_history[it])

In [89]:
normalize(data)
train_data, test_data = split(data)

In [90]:
#making numpy array of features and target
train_Y = train_data["Y"].to_numpy()
train_Y = train_Y.reshape(train_Y.shape[0],1)

train_X = train_data[["X1","X2"]].to_numpy()
print(train_X.shape)

(1155, 2)


In [103]:
train_X_terms, W = generate_vectors(train_X, 9)
W, cost_h = stochastic_gradient_descent_lasso(train_X_terms, train_Y, W+1, 1e-4, learning_rate=0.0001, iterations=20000)
errorloop(cost_h)

8666.058378891965
3946.274787693428
2303.836981570916
1425.9249053604628
1530.944467828883
980.3985914132307
998.964722078297
932.9265556718262
818.9074902265302
831.0000559410249
694.267023960154
704.243023214691
658.2954775328622
671.8305970937671
719.0709889573458
682.2218125977165
530.8233346486148
566.857088233342
672.8143707983032
652.1974491792558
601.0452369570177
648.8387506372069
453.98801166847545
491.0416270293399
475.89634622536965
528.424662648914
564.7803235207266
481.6718967280119
470.5773787693135
491.6721080214925
434.11047663113146
445.1923971738147
418.14293279195545
523.3577437246998
427.71159220844345
415.18615446611034
448.14756495030144
369.1926767081219
437.7728056862861
400.366792195987
346.1421442519782
362.29619656456157
409.81311101425536
317.98984309550434
378.68337168828265
409.7512268300029
415.004638806594
379.6548653267017
328.4542223855923
330.69553155312497
320.84734884170143
364.19754087757866
339.08052581670466
362.50374008410364
291.01615281418236